In [2]:
import pickle
with open(r'D:\ucl\pyssem\scenario-properties.pkl', 'rb') as file:
    scenario_properties = pickle.load(file)

from utils.drag.drag import JB2008_dens_func

In [47]:
# create a list of random decimal between 0 and 100
import random
import pandas as pd
import sympy as sp
from scipy.integrate import solve_ivp
random.seed(0)
random_numbers = [random.uniform(0, 100) for i in range(100000)]

h = scenario_properties.R0_km
scen_time_dates = scenario_properties.scen_times_dates
start_date = scenario_properties.start_date
end_date = scenario_properties.start_date + pd.DateOffset(years=scenario_properties.simulation_duration)
num_steps = scenario_properties.steps

In [36]:
import os
import json
import numpy as np
import pandas as pd
from scipy.spatial import KDTree

# Pre-load and preprocess density data
def preload_density_data(file_path):
    with open(file_path, 'r') as file:
        density_data = json.load(file)
    return density_data

# Precompute date mapping for given t values and date range
def precompute_date_mapping(start_date, end_date, num_points=101):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    total_days = (end_date - start_date).days
    dates = [start_date + pd.to_timedelta(i / (num_points - 1) * total_days, unit='d') for i in range(num_points)]
    date_mapping = [date.strftime('%Y-%m') for date in dates]
    return date_mapping

# Precompute nearest altitude mapping
def precompute_nearest_altitudes(available_altitudes, max_query=2000, resolution=1):
    altitude_tree = KDTree(np.array(available_altitudes).reshape(-1, 1))
    altitude_mapping = {}
    for alt in range(0, max_query + resolution, resolution):
        _, idx = altitude_tree.query([[alt]])
        nearest_alt = available_altitudes[idx[0]]
        altitude_mapping[alt] = nearest_alt
    return altitude_mapping

# Load data
density_data = preload_density_data(r'D:\ucl\pyssem\pyssem\utils\drag\dens_highvar_2000_dens_highvar_2000_lookup.json')
date_mapping = precompute_date_mapping(scenario_properties.start_date, end_date)
available_altitudes = list(map(int, list(density_data['2020-03'].keys())))
available_altitudes.sort()
nearest_altitude_mapping = precompute_nearest_altitudes(available_altitudes)

# Optimized function to use in solve_ivp
def JB2008_dens_func(t, h, density_data, date_mapping, nearest_altitude_mapping):
    """
    Calculate density at various altitudes based on a percentage through a time range
    using precomputed data for efficiency.

    :param t: Percentage of the way through the simulation, between 0 and 100
    :param h: List of altitudes for which densities are required
    :param density_data: Preloaded density data
    :param date_mapping: Precomputed date mapping
    :param nearest_altitude_mapping: Precomputed nearest altitude mapping
    :return: List of densities corresponding to each altitude in h
    """
    # Compute index for t assuming t is between 0 and 100
    t_index = int(t)
    target_date_str = date_mapping[t_index]

    # Check if the target date is in the dataset
    if target_date_str not in density_data:
        raise ValueError(f"No data available for {target_date_str}")

    # Compute densities for the requested altitudes
    density_values = []
    for alt in h:
        query_alt = min(alt, max(nearest_altitude_mapping.keys()))
        nearest_alt = nearest_altitude_mapping[query_alt]
        density_values.append(density_data[target_date_str][str(nearest_alt)])

    return density_values

for i in random_numbers:
    rho = JB2008_dens_func(i, h, density_data, date_mapping, nearest_altitude_mapping)

KeyboardInterrupt: 

In [50]:
scenario_properties.density_model

<function utils.drag.drag.static_exp_dens_func(t, h, species, scen_properties)>

In [45]:
# Lambdafiying the equations
symbols = scenario_properties.all_symbolic_vars

equations_flattened = [scenario_properties.equations[i, j] for j in range(scenario_properties.equations.cols) for i in range(scenario_properties.equations.rows)]
#drag_flattened = [scenario_properties.full_drag[i, j] for j in range(scenario_properties.full_drag.cols) for i in range(scenario_properties.full_drag.rows)]
#pmd_flattened = [scenario_properties.full_Cdot_PMD[i, j] for j in range(scenario_properties.full_Cdot_PMD.cols) for i in range(scenario_properties.full_Cdot_PMD.rows)]
#collisions_flattened = [scenario_properties.full_coll[i, j] for j in range(scenario_properties.full_coll.cols) for i in range(scenario_properties.full_coll.rows)]

# Convert equations to lambda functions
equations = [sp.lambdify(symbols, eq, 'numpy') for eq in equations_flattened]
#full_drag = [sp.lambdify(symbols, eq, 'numpy') for eq in drag_flattened]
#full_pmd = [sp.lambdify(symbols, eq, 'numpy') for eq in pmd_flattened]
#full_coll = [sp.lambdify(symbols, eq, 'numpy') for eq in collisions_flattened]

full_lambda_flattened = []

for i in range(len(scenario_properties.full_lambda)):
    if scenario_properties.full_lambda[i] is not None:
        full_lambda_flattened.extend(scenario_properties.full_lambda[i])
    else:
        # Append None to the list, length of scenario_properties.n_shells
        full_lambda_flattened.extend([None]*scenario_properties.n_shells)

In [54]:
scenario_properties.time_dep_density = True

In [56]:
from utils.simulation.scen_properties import population_shell
x0_flattened = scenario_properties.x0.T.values.flatten()
times = np.array(scenario_properties.scen_times)

# Same funciton but uses np.interp
def population_shell(t, N, full_lambda, equations):
    print(t)
    # Initialize the rate of change array
    dN_dt = np.zeros_like(N)

    if scenario_properties.time_dep_density:
        rho = JB2008_dens_func(t, h, density_data, date_mapping, nearest_altitude_mapping)
        rho_mat_k = np.repeat(rho[:-1], scenario_properties.species_length).reshape(-1, 1)
        rho_mat_kp1 = np.repeat(rho[1:], scenario_properties.species_length).reshape(-1, 1)

    for i in range(len(N)):
        if full_lambda[i] is not None:
            increase = np.interp(t, times, full_lambda[i])
            # If increase is nan set to 0
            if np.isnan(increase):
                dN_dt[i] += 0
            else:
                dN_dt[i] += increase

            # Compute the intrinsic rate of change from the differential equation
        dN_dt[i] += equations[i](*N)

    return dN_dt

output = solve_ivp(population_shell, [scenario_properties.scen_times[0], scenario_properties.scen_times[-1]], x0_flattened, 
                    args=(full_lambda_flattened, equations), 
                    t_eval=scenario_properties.scen_times, method='BDF')

0.0
7.676590391953218e-07
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.